In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from pathlib import Path

import nrrd
import numpy as np
import SimpleITK as sitk
import torch
from tqdm import tqdm

from utils import process_img_array
from matplotlib import pyplot as plt
from experiments.utils import generate_seg
from experiments.model_utils import load_medsam

device = 'cuda' if torch.cuda.is_available() else 'cpu'; print(device)
medsam = load_medsam("../work_dir/MedSAM/medsam_vit_b.pth", device)

In [ ]:
data_folder = Path("/media/liushifeng/KINGSTON/ULS Jan 2025/ULS23/novel_data/ULS23_DeepLesion3D")
output_folder = Path("/home/liushifeng/Documents/GitHub/MedSAM/experiments/outputs/extend_3d/ULS_up_down/")

In [ ]:
with open(data_folder / "train.txt") as f:
    train_names = [x.rstrip() for x in f.readlines()]
with open(data_folder / "val.txt") as f:
    val_names = [x.rstrip() for x in f.readlines()]

In [ ]:
for filename in tqdm(val_names[126:]):
    print(f"Segmenting {filename}")
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename
    break

In [ ]:
ct = sitk.ReadImage(img_path)
seg = sitk.ReadImage(seg_path)
ct_array = sitk.GetArrayFromImage(ct)
seg_array = sitk.GetArrayFromImage(seg)

In [ ]:
vol_seg = generate_seg(img_path, seg_path, medsam)

In [ ]:
np.where(vol_seg.sum(axis=(1,2)) > 0)

In [ ]:
for filename in tqdm(val_names[:1]):
    print(f"Segmenting {filename}")
    img_path = data_folder / "images" / filename
    seg_path = data_folder / "labels" / filename
    vol_seg = generate_seg(img_path, seg_path, medsam)

    # # save nrrd 3d seg for eval
    # nrrd.write(f"{output_folder}/{filename}_pred.nrrd", vol_seg)

In [ ]:
l, a = process_img_array(ct_array[60])